## Import Libraries

In [51]:
import pandas as pd
import pymysql

## Data Loading and Preprocessing

In [52]:
data = pd.read_csv("netflix_titles.csv")
data = pd.DataFrame(data)

data.dropna(inplace=True)

In [53]:
data['date_added'] = data['date_added'].str.strip()  # Remove extra spaces
data['date_added'] = pd.to_datetime(data['date_added'], format="%B %d, %Y", errors='coerce')

data['date_added'] = data['date_added'].dt.strftime('%Y-%m-%d %H:%M:%S')

data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24 00:00:00,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,2021-09-24 00:00:00,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24 00:00:00,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021-09-23 00:00:00,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,2021-09-21 00:00:00,1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...
...,...,...,...,...,...,...,...,...,...,...,...,...
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan",2016-03-09 00:00:00,2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20 00:00:00,2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01 00:00:00,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11 00:00:00,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


# Import to mysql

In [54]:
conn = pymysql.connect(
    host='mysql.clarksonmsda.org',
    port=3306,
    user='ia626',
    passwd='ia626clarkson',
    db='ia626',
    autocommit=True
)
cur = conn.cursor(pymysql.cursors.DictCursor)

In [55]:
def create_shows_table(cursor):
    cursor.execute("DROP TABLE IF EXISTS netflix_shows")
    create_table_query = """
        CREATE TABLE IF NOT EXISTS netflix_shows (
            show_id VARCHAR(50) PRIMARY KEY,
            type VARCHAR(50),
            title VARCHAR(255),
            director VARCHAR(255),
            cast TEXT,
            country VARCHAR(100),
            date_added DATE,
            release_year INT,
            rating VARCHAR(10),
            duration VARCHAR(50),
            listed_in VARCHAR(255),
            description TEXT
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1 AUTO_INCREMENT=1 ;
    """
    cursor.execute(create_table_query)
    print("Shows table created.")

In [56]:
def insert_data_to_mysql(cursor, dataframe, table_name="netflix_shows"):

    insert_query = f"""
        INSERT INTO {table_name} ( 	
        show_id, type, title, director, cast, country,
        date_added, release_year, rating, duration, listed_in, description
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    data_tuples = dataframe.to_records(index=False).tolist()
    
    try:
        cursor.executemany(insert_query, data_tuples)
        print(f"Successfully inserted {len(data_tuples)} records into {table_name}.")
    except Exception as e:
        print(f"Error inserting data: {e}")


In [57]:
def main():
    create_shows_table(cur)

    insert_data_to_mysql(cur, data)

    cur.close()
    conn.close()


if __name__ == "__main__":
    main()



Shows table created.
Successfully inserted 5332 records into netflix_shows.
